## Importing Libraries

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.window import Window

**1398. Customers Who Bought Products A and B but Not C (Medium)**

**Table: Customers**

| Column Name         | Type    |
|---------------------|---------|
| customer_id         | int     |
| customer_name       | varchar |

customer_id is the column with unique values for this table.
customer_name is the name of the customer.
 
**Table: Orders**

| Column Name   | Type    |
|---------------|---------|
| order_id      | int     |
| customer_id   | int     |
| product_name  | varchar |

order_id is the column with unique values for this table.
customer_id is the id of the customer who bought the product "product_name".
 
**Write a solution to report the customer_id and customer_name of customers who bought products "A", "B" but did not buy the product "C" since we want to recommend them to purchase this product.**

Return the result table ordered by customer_id.

The result format is in the following example.

**Example 1:**

**Input:** 

**Customers table:**

| customer_id | customer_name |
|-------------|---------------|
| 1           | Daniel        |
| 2           | Diana         |
| 3           | Elizabeth     |
| 4           | Jhon          |

**Orders table:**
| order_id   | customer_id  | product_name  |
|------------|--------------|---------------|
| 10         |     1        |     A         |
| 20         |     1        |     B         |
| 30         |     1        |     D         |
| 40         |     1        |     C         |
| 50         |     2        |     A         |
| 60         |     3        |     A         |
| 70         |     3        |     B         |
| 80         |     3        |     D         |
| 90         |     4        |     C         |

**Output:** 
| customer_id | customer_name |
|-------------|---------------|
| 3           | Elizabeth     |

**Explanation:** Only the customer_id with id 3 bought the product A and B but not the product C.

In [0]:
customers_data_1398 = [
    (1, "Daniel"),
    (2, "Diana"),
    (3, "Elizabeth"),
    (4, "Jhon")
]

customers_columns_1398 = ["customer_id", "customer_name"]
customers_df_1398 = spark.createDataFrame(customers_data_1398, customers_columns_1398)
customers_df_1398.show()

orders_data_1398 = [
    (10, 1, "A"),
    (20, 1, "B"),
    (30, 1, "D"),
    (40, 1, "C"),
    (50, 2, "A"),
    (60, 3, "A"),
    (70, 3, "B"),
    (80, 3, "D"),
    (90, 4, "C")
]

orders_columns_1398 = ["order_id", "customer_id", "product_name"]
orders_df_1398 = spark.createDataFrame(orders_data_1398, orders_columns_1398)
orders_df_1398.show()


+-----------+-------------+
|customer_id|customer_name|
+-----------+-------------+
|          1|       Daniel|
|          2|        Diana|
|          3|    Elizabeth|
|          4|         Jhon|
+-----------+-------------+

+--------+-----------+------------+
|order_id|customer_id|product_name|
+--------+-----------+------------+
|      10|          1|           A|
|      20|          1|           B|
|      30|          1|           D|
|      40|          1|           C|
|      50|          2|           A|
|      60|          3|           A|
|      70|          3|           B|
|      80|          3|           D|
|      90|          4|           C|
+--------+-----------+------------+



In [0]:
customer_products_df_1398 = orders_df_1398\
                                .groupBy("customer_id") \
                                    .agg(collect_set("product_name").alias("products"))

In [0]:
customer_products_df_1398\
                .filter(
                    array_contains(col("products"),"A") &
                    array_contains(col("products"),"B") &
                    (~array_contains(col("products"),"C"))
                )\
                .join(customers_df_1398, on="customer_id") \
                    .select("customer_id", "customer_name") \
                        .orderBy("customer_id").show()

+-----------+-------------+
|customer_id|customer_name|
+-----------+-------------+
|          3|    Elizabeth|
+-----------+-------------+

